In [ ]:

! pip install datasets
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from datasets import load_dataset
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics import f1_score
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


In [5]:
from datasets import load_dataset


In [6]:
from string import punctuation

import matplotlib.pyplot as plt
from nltk.tokenize import ToktokTokenizer
import numpy as np
from tqdm import tqdm
import torch
import seaborn as sns

In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [8]:
import os
import gensim
from gensim import models
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import word2vec


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np

In [10]:
from tensorflow.keras.optimizers import SGD


In [11]:
from keras.layers import BatchNormalization
from keras import layers


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random

In [13]:
import tensorflow as tf


*Загрузка нужного датасета*

In [15]:
dataset_path = "tweet_eval"
dataset_name = "sentiment"


In [16]:
train_dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")
valid_dataset = load_dataset(path=dataset_path, name=dataset_name, split="validation")
test_dataset = load_dataset(path=dataset_path, name=dataset_name, split="test")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


In [17]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [18]:
for batch in train_loader:
    break

batch

{'text': ['"Nikki Bella may have the Divas title, but she\'ll never have a child with John Cena."',
  '"Only 2 hours left! Gonna be so sick, concert, bachelor party and I get paid to not work on Monday.As Ice Cube would say today was a good day"'],
 'label': tensor([1, 2])}

In [19]:
punct = punctuation + "«»"


In [20]:
tokenizer = ToktokTokenizer()


In [21]:
def load_embeddings(file_path, pad_token='PAD', unk_token='UNK', max_words=100_000, verbose=True):
    
    vocab = dict()
    embeddings = list()

    with open(file_path) as file_object:

        vocab_size, embedding_dim = file_object.readline().strip().split()

        vocab_size = int(vocab_size)
        embedding_dim = int(embedding_dim)

        # в файле 1 000 000 слов с векторами, давайте ограничим для простоты этот словарь
        max_words = vocab_size if max_words <= 0 else max_words

        # добавим пад токен и эмбеддинг в нашу матрицу эмбеддингов и словарь
        vocab[pad_token] = 0
        embeddings.append(np.zeros(embedding_dim))

        # добавим унк токен и эмбеддинг в нашу матрицу эмбеддингов и словарь
        vocab[unk_token] = 1
        embeddings.append(np.ones(embedding_dim))

        progress_bar = tqdm(total=max_words, disable=not verbose, desc='Reading embeddings file')

        for line in file_object:
            parts = line.strip().split()

            token = ' '.join(parts[:-embedding_dim]).lower()

            if token in vocab:
                continue

            word_vector = np.array(list(map(float, parts[-embedding_dim:])))

            vocab[token] = len(vocab)
            embeddings.append(word_vector)

            progress_bar.update()

            if len(vocab) == max_words:
                break

        progress_bar.close()

    embeddings = np.stack(embeddings)
    
    return vocab, embeddings

In [26]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [27]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

tokenizer.tokenize('@user when a father is dysfunctional and is')

['user', 'when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is']

In [28]:
def message_to_token_list(s):
  tokens = tokenizer.tokenize(s)
  lowercased_tokens = [t.lower() for t in tokens]
  lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
  useful_tokens = [t for t in lemmatized_tokens if t in words]

  return useful_tokens

message_to_token_list('@user feet a fathers is dysfunctional and is')

[]

In [29]:
def message_to_word_vectors(message, word_dict=words):
  processed_list_of_tokens = message_to_token_list(message)

  vectors = []

  for token in processed_list_of_tokens:
    if token not in word_dict:
      continue
    
    token_vector = word_dict[token]
    vectors.append(token_vector)
  
  return np.array(vectors, dtype=float)

In [30]:
message_to_word_vectors('@user when a father is dysfunctional and is')

array([], dtype=float64)

In [31]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [32]:
model = word2vec.Word2Vec(train_dataset['text'], size=300, window=3, workers=4)


In [33]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [34]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [35]:
train_mean=mean_vectorizer(w2v).fit(train_dataset['text']).transform(train_dataset['text'])
train_mean.shape

(45615, 300)

In [36]:
import numpy

In [37]:
arr = numpy.array(train_mean)

In [ ]:
train_dataset['label']

In [39]:
train_mean

array([[ 4.1400813e-02,  1.2001749e-01,  2.2238474e-01, ...,
        -1.2297049e-02,  2.2753935e-04, -1.9730164e-01],
       [ 2.4544667e-02,  6.6501953e-02,  1.7950162e-01, ...,
        -2.8134530e-02, -3.1445410e-02, -1.6728324e-01],
       [ 1.5022709e-02,  1.6172068e-01,  2.8623390e-01, ...,
        -2.2065487e-02,  7.6562287e-03, -2.4499306e-01],
       ...,
       [ 6.1732080e-02,  1.2619582e-01,  2.4854836e-01, ...,
         1.8510440e-03, -2.4066272e-03, -1.8180101e-01],
       [ 2.8220890e-02,  1.2065917e-01,  2.8205311e-01, ...,
        -1.1627383e-02, -2.2617925e-02, -2.5161907e-01],
       [ 5.3740535e-02,  9.8537773e-02,  2.2282885e-01, ...,
         6.6616642e-03, -2.3685217e-02, -8.4446251e-02]], dtype=float32)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [46]:
X = arr
y = label_arr

In [45]:
label_arr

array([2, 1, 1, ..., 2, 1, 1])

In [44]:
label_arr = numpy.array(train_dataset['label'])


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .98)

In [48]:
Input = keras.Input(shape=(300,))
l1 = layers.Dense(1024, activation='relu')(Input)
l1 = layers.Dropout(0.025)(l1)
l1 = layers.BatchNormalization()(l1)

l1 = layers.Dense(512, activation='relu')(l1)
l1 = layers.Dropout(0.035)(l1)
l1 = layers.BatchNormalization()(l1)

l2 = layers.Dense(512, activation='relu')(Input)
l2 = layers.Dropout(0.045)(l2)
l2 = layers.BatchNormalization()(l2)

add = layers.Add()([l1, l2])
l3 = layers.Dense(256, activation='relu')(add)
l3 = layers.BatchNormalization()(l3)
final=layers.Dense(3,activation='softmax')(l3)

model = keras.Model(Input, final)


In [49]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)


In [50]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [52]:
model.fit(X_train, y_train,
          epochs=40,
          batch_size=1024*5)

Epoch 1/40
9/9 [==============================] - 7s 769ms/step - loss: 0.8800 - accuracy: 0.5788
Epoch 2/40
9/9 [==============================] - 7s 765ms/step - loss: 0.8679 - accuracy: 0.5864
Epoch 3/40
9/9 [==============================] - 7s 756ms/step - loss: 0.8599 - accuracy: 0.5898
Epoch 4/40
9/9 [==============================] - 7s 766ms/step - loss: 0.8552 - accuracy: 0.5919
Epoch 5/40
9/9 [==============================] - 7s 762ms/step - loss: 0.8520 - accuracy: 0.5962
Epoch 6/40
9/9 [==============================] - 7s 757ms/step - loss: 0.8469 - accuracy: 0.5983
Epoch 7/40
9/9 [==============================] - 7s 759ms/step - loss: 0.8437 - accuracy: 0.6005
Epoch 8/40
9/9 [==============================] - 7s 763ms/step - loss: 0.8394 - accuracy: 0.6042
Epoch 9/40
9/9 [==============================] - 7s 763ms/step - loss: 0.8346 - accuracy: 0.6049
Epoch 10/40
9/9 [==============================] - 7s 761ms/step - loss: 0.8325 - accuracy: 0.6094
Epoch 11/40
9/9 [==